# Agent Component: Rainfall Agent

The `RainfallAgent` is a type of **Disturbance Agent**. Its purpose is to inject an external event into the simulation to test the system's resilience and response.

This agent is configured with a specific time window (`start_time`, `duration`) and an `inflow_rate`. When the simulation's current time falls within this window, the agent will continuously publish a message containing the inflow rate to a specified topic. A physical component, like a `Reservoir`, can subscribe to this topic to have its own inflow updated with this disturbance value.

## Demonstration

The simulation below demonstrates the `RainfallAgent`'s behavior in isolation. We will create the agent and a listener. By calling the agent's `run()` method at different time steps, we can see that it only publishes its message during its configured active period.

In [ ]:
from swp.simulation_identification.disturbances.rainfall_agent import RainfallAgent
from swp.central_coordination.collaboration.message_bus import MessageBus

# 1. Setup
message_bus = MessageBus()
DISTURBANCE_TOPIC = "disturbance.catchment.inflow"

rainfall_config = {
    "topic": DISTURBANCE_TOPIC,
    "start_time": 10,
    "duration": 5,
    "inflow_rate": 150.0
}

rainfall_agent = RainfallAgent("weather_event_1", message_bus, rainfall_config)

# 2. Create a listener
captured_messages = []
def listener(message):
    captured_messages.append(message)

message_bus.subscribe(DISTURBANCE_TOPIC, listener)

# 3. Simulation Loop
print("--- Running Simulation ---")
for t in range(20):
    print(f"Time = {t}s")
    rainfall_agent.run(current_time=t)

print("\n--- Simulation Finished ---")

## Results

Below are the messages captured by our listener. They were only sent between t=10 and t=14 (inclusive), which corresponds to the configured `start_time` of 10 and `duration` of 5. The agent correctly remained silent at all other times.

In [ ]:
print(f"Captured {len(captured_messages)} messages:")
for i, msg in enumerate(captured_messages):
    print(f"  Message {i+1}: {msg}")